In [1]:
#Import Dependencies
import datetime as dt
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify

In [2]:
# reflect an existing database into a new model
engine = create_engine("sqlite:///hawaii.sqlite")
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [3]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [4]:
# We can view all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [5]:
#session link
session = Session(engine)

In [ ]:
#set up flask
app = Flask(__name__)

#define welcome route
@app.route("/")
def welcome():
    return(
    '''
    Welcome to the Climate ANalysis API!
    Available Routes:
    /api/v1.0/precipitation
    /api/v1.0/stations
    /api/v1.0/tobs
    /api/v1.0/temp/start/end
    ''')

#precipitation route
@app.route("/api/v1.0/precipitation")
def precipitation():
   prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
   precipitation = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date >= prev_year).all()
   precip = {date: prcp for date, prcp in precipitation}
   return jsonify(precip)

#stations route
@app.route("/api/v1.0/stations")
def stations():
    results = session.query(Station.station).all()
    stations = list(np.ravel(results))
    return jsonify(stations=stations)

#monthly temps route
@app.route("/api/v1.0/tobs")
def temp_monthly():
    prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
    results = session.query(Measurement.tobs).\
        filter(Measurement.station == 'USC00519281').\
        filter(Measurement.date >= prev_year).all()
    temps = list(np.ravel(results))
    return jsonify (temps = temps)

#statistics route
@app.route("/api/v1.0/temp/<start>")
@app.route("/api/v1.0/temp/<start>/<end>")
def stats(start=None, end=None):
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]

    if not end:
        results = session.query(*sel).\
            filter(Measurement.date >= start).all()
        temps = list(np.ravel(results))
        return jsonify(temps)

    results = session.query(*sel).\
        filter(Measurement.date >= start).\
        filter(Measurement.date <= end).all()
    temps = list(np.ravel(results))
    return jsonify(temps)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Jun/2022 13:36:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2022 13:36:46] "GET /api/v1.0/temp/start/end%20route HTTP/1.1" 200 -
[2022-06-18 13:37:04,373] ERROR in app: Exception on /api/v1.0/temp/2017-06-01/2017-06-30 [GET]
Traceback (most recent call last):
  File "C:\Users\eliza\anaconda3\lib\site-packages\sqlalchemy\engine\base.py", line 1708, in _execute_context
    context = constructor(
  File "C:\Users\eliza\anaconda3\lib\site-packages\sqlalchemy\engine\default.py", line 1013, in _init_compiled
    self.cursor = self.create_cursor()
  File "C:\Users\eliza\anaconda3\lib\site-packages\sqlalchemy\engine\default.py", line 1361, in create_cursor
    return self.create_default_cursor()
  File "C:\Users\eliza\anaconda3\lib\site-packages\sqlalchemy\engine\default.py", line 1364, in create_default_cursor
    return self._dbapi_connection.cursor()
  File "C:\Users\eliza\anaconda3\lib\site-packages\sqlalchem